# Mixtures of MNLs

In [424]:
import gurobipy as gp

In [425]:
import numpy as np
import numpy.random as npr
N = 500 #5 #25 50 75 100
M = 20 #1 5 10 20
v = npr.rand((N+1), M)
for j in range(M):
    v[:, j] = v[:, j] / v[0, j]
v = v[1:][:]
theta = list(np.random.dirichlet(np.ones(M), size=1)[0])

In [426]:
def IP_mmnl(p):
    global v, N
    global theta, M

    #create a new model
    model = gp.Model("IP_mmnl")

    # create decision variables and store them in the arrays z, x, y
    z = model.addVars(M, vtype=gp.GRB.CONTINUOUS, name="z")
    x = model.addVars(N, M, vtype=gp.GRB.CONTINUOUS, lb=0, name="x")
    y = model.addVars(N, vtype=gp.GRB.BINARY, name="y")
    model.update()

    # set objective function
    objExpr = gp.LinExpr()
    for j in range(M):
        objExpr += theta[j] * z[j]
    model.setObjective(objExpr, gp.GRB.MAXIMIZE)

    B = max(p)
    # add constraints
    for j in range(M):
        myConstr = gp.LinExpr()
        for i in range(N):
            myConstr += v[i][j] * x[i, j]
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = y[i]*B, name = "x_1st_constraint_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.GREATER_EQUAL, rhs = p[i] - z[j] - (1-y[i])*B, name = "x_2nd_constraint_" + str(i) + '_' + str(j))
            model.addConstr(lhs = x[i, j], sense = gp.GRB.LESS_EQUAL, rhs = p[i] - z[j] + (1-y[i])*B, name = "x_3rd_constraint_" + str(i) + '_' + str(j))
        model.addConstr(lhs = z[j], sense = gp.GRB.LESS_EQUAL, rhs = myConstr, name = "z_" + str(j))

    # write model 
    model.write("IP_mmnl.lp")

    # solve model
    model.Params.OutputFlag = 0
    model.optimize()
    
    bestS = []
    profit = float(model.objVal)
    for i in range(N):
        if y[i].x == 1:
            bestS.append(i)
    bestS = sorted(bestS)
    return profit, bestS

In [427]:
def profit_mmnl(p, S):
    global v
    global theta, M
    pi = 0
    for j in range(M):
        Vj_S = 0
        num = 0
        for i in S:
            num += p[i]*v[i][j]
            Vj_S += v[i][j]
        pi += theta[j]*num/(1+Vj_S)
    return pi

In [428]:
def brute_force(p):
    global N
    # compute the profit of each assortment
    best_profit = 0
    best_S = []
    for i in range(2**N):
        binary = np.binary_repr(i, width=N)
        assortment = [int(x) for x in binary]
        S = []
        for j in range(N):
            if assortment[j] == 1:
                S.append(j)
        if best_profit < profit_mmnl(p, S):
            best_profit = profit_mmnl(p, S)
            best_S = S
    return best_profit, best_S

In [429]:
def nested_by_price_mmnl(p):
    global N
    profits = [0 for i in range(N)]
    S = []
    for i in range(N):
        S.append(i)
        profits[i] = profit_mmnl(p, S)
    best_S = [i for i in range(np.argmax(profits)+1)]
    return max(profits), best_S

In [430]:
def find_best_option_mmnl(p, S):
    global v, N
    current_best = profit_mmnl(p, S)
    bestS = S
    flag = 0
    N_S = [k for k in range(N) if k not in S]
    for k in N_S:
        newS = S + [k]
        if profit_mmnl(p, newS) >= current_best:
            flag = 1
            bestS = newS
            current_best = profit_mmnl(p, newS)
    return current_best, bestS, flag

def greedy_mmnl(p):
    global v, N
    max_profit = -1
    best_S = []
    S = []
    flag = 1
    while flag==1:
        max_profit, S, flag = find_best_option_mmnl(p, S)
    best_S = sorted(S)
    return max_profit, best_S

In [431]:
import matplotlib.pyplot as plt
import math
import statistics as st
import time

In [432]:
n_simulations = 1
avg_price = 1
std_price = 0.5
#compute prices
psim = [0 for k in range(n_simulations)]
for k in range(n_simulations):
    psim[k] = abs(npr.normal(avg_price, std_price, N))
    # p[i] = npr.uniform(0, 2*avg_price, N)
    # p[i] = npr.exponential(avg_price, N)
    psim[k] = sorted(psim[k], reverse=True)

In [433]:
# problem solved using the MIP formulation
profits_IP = []
times_IP = []
bestS_IP = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = IP_mmnl(psim[k])
    end_time = time.time()
    profits_IP.append(profit)
    times_IP.append(end_time-start_time)
    bestS_IP.append(bestS)
avg_time_IP = st.mean(times_IP)

In [434]:
# problem solved using the nested formulation
profits_nested = []
times_nested = []
bestS_nested = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = nested_by_price_mmnl(psim[k])
    end_time = time.time()
    profits_nested.append(profit)
    times_nested.append(end_time-start_time)
    bestS_nested.append(bestS)
avg_time_nested = st.mean(times_nested)

In [435]:
# problem solved using the greedy formulation
profits_greedy = []
times_greedy = []
bestS_greedy = []
for k in range(n_simulations):
    start_time = time.time()
    profit, bestS = greedy_mmnl(psim[k])
    end_time = time.time()
    profits_greedy.append(profit)
    times_greedy.append(end_time-start_time)
    bestS_greedy.append(bestS)
avg_time_greedy = st.mean(times_greedy)

In [436]:
# counting how many instances are different
count_nested = 0
different_results_nested = []
count_greedy = 0
different_results_greedy = []
for k in range(n_simulations):
    if bestS_IP[k] != bestS_greedy[k]:
        count_greedy += 100/n_simulations
        different_results_greedy.append(100*profits_greedy[k]/profits_IP[k])
    print(bestS_IP, bestS_greedy)
    if bestS_IP[k] != bestS_nested[k]:
        count_nested += 100/n_simulations
        different_results_nested.append(100*profits_nested[k]/profits_IP[k])
    print(bestS_IP, bestS_nested)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]] [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]] [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]]


In [437]:
if different_results_greedy != []:
  print('The greedy algorithm is different from the MIP in ' + str(count_greedy) + ' of the instances' , \
        'with an average ratio of ' + str(st.mean(different_results_greedy)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_greedy)) + '%', \
                'and an average time of ' + str(avg_time_greedy) + ' seconds')
if different_results_nested != []:
  print('The nested algorithm is different from the MIP in ' + str(count_nested) + ' of the instances', \
        'with an average ratio of ' + str(st.mean(different_results_nested)) + '%', \
              'and a minimum ratio of ' + str(min(different_results_nested)) + '%', 
                'and an average time of ' + str(avg_time_nested) + ' seconds')

In [438]:
# brute_force(psim[0])
